#### Initial imports

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

#### Set processing parameters

In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
USE_PARALLEL = False

### Prep lists of input files

In [3]:
from glob import glob
import re
import pandas as pd
from random import sample, seed

In [4]:
encoder_list = ['resnet34']
avail_suffix = ['rgb', 'tpi', 'shade', 'ndvi', 'dsm']

In [5]:
# set working directory
#os.chdir('/project/cper_neon_aop/cper_pdog_uas')

# set directories for training data and labels
DATA_FOLDER = './cnn_train_images/{}_{}.tif'
LABEL_FOLDER = './cnn_train_labels/{}_labels.tif'

In [6]:
# read in csvs with training information
df_tiles = pd.read_csv('train_tiles/train_bboxes_all_assigned.csv')
df_polys = pd.read_csv('train_polys/train_polys_all.csv')

In [7]:
# get all ids to be used
label_files = glob(LABEL_FOLDER.replace('{}', '*'))
all_ids = [re.sub('_labels.tif', '', os.path.basename(f)) for f in label_files]
all_tiles = list(set(['_'.join(y.split('_')[2:]) for y in all_ids]))

In [8]:
# separate training and test data and get paths to files
all_files = glob(DATA_FOLDER.replace('{}', '*'))
all_train_tiles = [x for x in df_tiles.apply(lambda x: '_'.join([x.Pasture, x.Tile]) if x.Train == 1 else '', axis=1) if x != '' and x in all_tiles]
test_tiles = list(set(all_tiles) - set(all_train_tiles))

all_train_ids = [x for x in all_ids if '_'.join(x.split('_')[-3:]) in all_train_tiles]
test_ids = list(set(all_ids) - set(all_train_ids))

seed(321)
valid_ids = sample(all_train_ids, int(np.ceil(len(all_train_ids)*0.3)))
train_ids = list(set(all_train_ids) - set(valid_ids))

train_files = [f for f in all_files if '_'.join(os.path.basename(f).split('_')[:-1]) in train_ids]
valid_files = [f for f in all_files if '_'.join(os.path.basename(f).split('_')[:-1]) in valid_ids]
test_files = [f for f in all_files if '_'.join(os.path.basename(f).split('_')[:-1]) in test_ids]

In [9]:
tile_ids = df_tiles[(df_tiles['trainer'] != 'Nick') &
                    (df_tiles['Digitize'] == 1)].apply(lambda x: '_'.join([x.Pasture, x.Tile]), axis=1)
#all_tiles#
[x for x in all_tiles if x not in tile_ids.to_list()]

[]

In [10]:
[x for x in tile_ids.to_list() if x not in all_tiles]

[]

### Dataloader
Writing helper class for data extraction, tranformation and preprocessing

In [11]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
from skimage import io

In [12]:
class Dataset(BaseDataset):
    """Read images, apply augmentation and preprocessing transformations.
    
    Args:
        ids (list): list of unique ids for all images
        images_path (str): path to data images
        masks_path (str): path to label masks
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    
    CLASSES = ['other', 'burrow']
    
    def __init__(
            self, 
            ids,
            suffix_list,
            images_path,
            masks_path, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            suffix_dict = {
        'rgb': {'channels': 3,
                'dtype': 'uint8'},
        'tpi': {'channels': 1,
                'dtype': 'float32'},
        'dsm': {'channels': 1,
                'dtype': 'float32'},
        'shade': {'channels': 1,
                  'dtype': 'float32'},
        'ndvi': {'channels': 1,
                  'dtype': 'float32'}
    }
    ):
        # get IDs as attribute
        self.ids = ids
        
        # get suffix info
        self.suffix_dict = suffix_dict
        
        # get list of suffixes as attribute
        self.suffix_list = suffix_list
        
        # List of files
        self.images_fps = []
        self.masks_fps = [masks_path.format(id) for id in ids]
        for id in ids:
            self.images_fps.append({s: images_path.format(id, s) for s in suffix_list})
            
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image_list = []
        self.image_dict = {}
        for s in self.suffix_list:
            image_s = np.asarray(io.imread(self.images_fps[i][s]), dtype=self.suffix_dict[s]['dtype'])
            if len(image_s.shape) == 2:
                image_s = np.expand_dims(image_s, axis=-1)
            image_s[np.isnan(image_s)] = 0
            if s == 'ndvi':
                image_s[image_s < 0] = 0
            if self.preprocessing:
                image_s = normalize_fn(image_s, s, image_stats)
            image_list.append(image_s)
            self.image_dict[s] = image_s
        if len(image_list) == 1:
            image = image_list[0]
        else:
            image = np.concatenate(image_list, axis=-1)
        mask = np.asarray(io.imread(self.masks_fps[i]), dtype='float32')
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1)#.astype('float32')
        #print('fetched: ', self.ids[i])
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # convert final image arrays to tensors
        image = torch.from_numpy(image.transpose(2, 0, 1).astype('float32'))
        mask = torch.from_numpy(mask.transpose(2, 0, 1).astype('float32'))
        return image, mask
        
    def __len__(self):
        return len(self.ids)




### Augmentations

In [13]:
import albumentations as albu
import random

In [14]:
def get_training_augmentation():
    win_size = 32 * random.randint(7, 10)
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),

        #albu.ShiftScaleRotate(scale_limit=0.0, rotate_limit=45, shift_limit=0.1, p=1, border_mode=0),

        #albu.PadIfNeeded(min_height=win_size, min_width=win_size, always_apply=True, border_mode=4),
        albu.RandomCrop(height=win_size, width=win_size, always_apply=True),

        #albu.GaussNoise(p=0.2, var_limit=1.0),
        #albu.Perspective(p=0.5),

        #albu.OneOf(
        #    [
        #        #albu.CLAHE(p=1), # required int8 images
        #        albu.RandomBrightnessContrast(p=1),
        #        #albu.RandomGamma(p=1),
        #        #albu.HueSaturationValue(p=1),
        #    ],
        #    p=0.9,
        #),

        albu.OneOf(
            [
                albu.Sharpen(p=1),
                albu.Blur(blur_limit=(3, 7), p=1),
                albu.MotionBlur(blur_limit=(3, 7), p=1),
            ],
            p=0.25,
        ),
    ]
    return albu.Compose(train_transform)

### Create preprocessing function from training data stats

In [15]:
import torch
import numpy as np

In [16]:
CLASSES = ['burrow']

In [17]:
image_stats = {
    'rgb': {'min': 0.0,
            'max': 255.0},
    'ndvi': {'min': 0.0,
             'max': 1.0}
}

In [18]:
for suffix_list in [['tpi'], ['shade']]:
    train_dataset = Dataset(
        train_ids,
        suffix_list,
        DATA_FOLDER,
        LABEL_FOLDER,
        classes=CLASSES)
    #train_dataset[0][0].cpu().numpy().shape
    min_list = []
    max_list = []
    for i in range(len(train_dataset)):
        min_list.append(np.min(train_dataset[i][0].cpu().numpy()))
        max_list.append(np.max(train_dataset[i][0].cpu().numpy()))
    image_stats[suffix_list[0]] = {
        'min': np.min(min_list),
        'max': np.max(max_list)
    }

In [19]:
df_image_stats = pd.DataFrame(image_stats).reset_index().rename(
    columns={'index': 'stat'})
df_image_stats.to_csv('./_utils/image_stats.csv', index=False)

In [20]:
# create function to normalize all data in range 0-1
def normalize_fn(image, image_suffix, stats_dict):
    if image_suffix in stats_dict.keys():
        min_tmp = stats_dict[image_suffix]['min']
        max_tmp = stats_dict[image_suffix]['max']
    else:
        # normalize to individual image if min/max stats not specified in dictionary
        min_tmp = np.min(image)
        max_tmp = np.max(image)
    return (image - min_tmp) / (max_tmp - min_tmp)
    
        

In [21]:
suffix_list = ['ndvi']
train_dataset = Dataset(
        train_ids,
        suffix_list,
        DATA_FOLDER,
        LABEL_FOLDER,
        #augmentation=get_training_augmentation(),
        preprocessing=True,
        classes=CLASSES)
min_list_tmp = []
for i in range(len(train_dataset)):
    min_list_tmp.append(np.min(train_dataset[i][0].cpu().numpy()))

In [22]:
np.min(min_list_tmp)

0.0

### Create and train model

In [23]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils
import torch.nn as nn
import gc

In [24]:
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cuda'# 'cpu'# 

In [25]:
import itertools
#list(itertools.combinations(avail_suffix, len(avail_suffix)))
suffix_combinations = list()
for n in range(1, len(avail_suffix) + 1):
    suffix_combinations += list(itertools.combinations(avail_suffix, n))

In [26]:
for mod in ['deeplabplus', 'fpn', 'manet', 'unetplus']:
    print('\n\n----------------------------------------------------------')
    print('----------------------------------------------------------')
    print('Now running model: ' + mod)
    print('----------------------------------------------------------')
    outDIR = './cnn_results_' + mod + '/'
    if not os.path.exists(outDIR):
        os.mkdir(outDIR)
    for suffix_sub in suffix_combinations:
        suffix_list = list(suffix_sub)
        print('\n\n----------------------------------------------------------')
        print(suffix_list)
        if os.path.exists(outDIR + 'best_model_' + '_'.join(suffix_list) + '_validation.txt'):
            print('skipping - already trained.')
            continue
        else:
            train_dataset = Dataset(
                train_ids,
                suffix_list,
                DATA_FOLDER,
                LABEL_FOLDER,
                augmentation=get_training_augmentation(),
                preprocessing=True,
                classes=CLASSES)

            valid_dataset = Dataset(
                valid_ids,
                suffix_list,
                DATA_FOLDER,
                LABEL_FOLDER,
                #augmentation=get_validation_augmentation(),
                preprocessing=True,
                classes=CLASSES)


            train_loader = DataLoader(train_dataset, batch_size=6, shuffle=False,
                                      drop_last=True, num_workers=6, pin_memory=False)
            valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=False)

            # create segmentation model with pretrained encoder
            if mod == 'deeplabplus':
                model = smp.DeepLabV3Plus(
                    encoder_name=ENCODER, 
                    encoder_weights=ENCODER_WEIGHTS, 
                    classes=len(CLASSES), 
                    activation=ACTIVATION,
                    in_channels=train_dataset[0][0].shape[0],
                )
            elif mod == 'fpn':
                model = smp.FPN(
                    encoder_name=ENCODER, 
                    encoder_weights=ENCODER_WEIGHTS, 
                    classes=len(CLASSES), 
                    activation=ACTIVATION,
                    in_channels=train_dataset[0][0].shape[0],
                )
            elif mod == 'manet':
                model = smp.MAnet(
                    encoder_name=ENCODER, 
                    encoder_weights=ENCODER_WEIGHTS, 
                    classes=len(CLASSES), 
                    activation=ACTIVATION,
                    in_channels=train_dataset[0][0].shape[0],
                )
            elif mod == 'unetplus':
                model = smp.UnetPlusPlus(
                    encoder_name=ENCODER, 
                    encoder_weights=ENCODER_WEIGHTS, 
                    classes=len(CLASSES), 
                    activation=ACTIVATION,
                    in_channels=train_dataset[0][0].shape[0],
                )
            else:
                print('ERROR: model "' + mod + '" not found!')
                break
                
            if USE_PARALLEL:
                model = nn.DataParallel(model)

            preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

            # Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
            # IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

            loss = smp.losses.MCCLoss()
            loss.__name__ = 'mccloss'
            metrics = [
                utils.metrics.IoU(threshold=0.5),
                utils.metrics.Accuracy(threshold=0.5),
                utils.metrics.Precision(threshold=0.5),
                utils.metrics.Recall(threshold=0.5),
                utils.metrics.Fscore(threshold=0.5)
            ]

            optimizer = torch.optim.Adam([ 
                dict(params=model.parameters(), lr=0.0001),
            ])

            # create epoch runners 
            # it is a simple loop of iterating over dataloader`s samples
            train_epoch = utils.train.TrainEpoch(
                model, 
                loss=loss, 
                metrics=metrics, 
                optimizer=optimizer,
                device=DEVICE,
                verbose=True,
            )

            valid_epoch = utils.train.ValidEpoch(
                model, 
                loss=loss, 
                metrics=metrics, 
                device=DEVICE,
                verbose=True,
            )

            # train model for up to 60 epochs

            max_score = 0
            max_score_train = 0
            no_improve = 0

            for i in range(1, 61):

                print('\nEpoch: {}'.format(i))
                train_logs = train_epoch.run(train_loader)
                valid_logs = valid_epoch.run(valid_loader)

                # do something (save model, change lr, etc.)
                if max_score < valid_logs['fscore']:
                    max_score = valid_logs['fscore']
                    max_score_train = train_logs['fscore']
                    torch.save(model, outDIR + 'best_model_' + '_'.join(suffix_list) + '.pth')
                    valid_logs['best_epoch'] = i
                    valid_logs['resolution'] = 1.5
                    best_valid_logs = valid_logs.copy()
                    print('Model saved!')
                    no_improve = 0
                else:
                    no_improve += 1
                    print('No improvement in ' + str(no_improve) + ' epochs. Model not saved.')

                if i > 15:
                    if no_improve >= 5:
                        if (train_logs['fscore'] - max_score_train) < 0.05:
                            print('More than 5 epochs without validation improvement while learning rate <= 1e-5 and training improvement < 0.05...ending training')
                            with open(outDIR + 'best_model_' + '_'.join(suffix_list) + '_validation.txt','w') as data: 
                                data.write(str(best_valid_logs))
                            break
                        elif (train_logs['fscore'] - max_score_train) < 0.15 and no_improve == 15:
                            print('15 epochs without validation improvement while learning rate <= 1e-5...ending training')
                            with open(outDIR + 'best_model_' + '_'.join(suffix_list) + '_validation.txt','w') as data: 
                                data.write(str(best_valid_logs))
                            break
                if i == 60:
                    with open(outDIR + 'best_model_' + '_'.join(suffix_list) + '_validation.txt','w') as data: 
                                data.write(str(best_valid_logs))

                if i%15 == 0:
                    no_improve = 0
                    optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.1
                    print('Decrease decoder learning rate by factor of 10')

            del model, train_epoch, valid_epoch
            gc.collect()
            torch.cuda.empty_cache()



----------------------------------------------------------
----------------------------------------------------------
Now running model: deeplabplus
----------------------------------------------------------


----------------------------------------------------------
['rgb']

Epoch: 1
train: 100%|█| 51/51 [00:07<00:00,  6.85it/s, mccloss - 0.8849, iou_score - 0.07467, accuracy - 0.6396, precision - 0.07
valid: 100%|█| 133/133 [00:01<00:00, 71.34it/s, mccloss - 0.9457, iou_score - 0.1789, accuracy - 0.7831, precision - 0.1
Model saved!

Epoch: 2
train: 100%|█| 51/51 [00:04<00:00, 11.95it/s, mccloss - 0.7953, iou_score - 0.1476, accuracy - 0.8415, precision - 0.149
valid: 100%|█| 133/133 [00:01<00:00, 70.79it/s, mccloss - 0.9234, iou_score - 0.5652, accuracy - 0.9635, precision - 0.6
Model saved!

Epoch: 3
train: 100%|█| 51/51 [00:04<00:00, 11.96it/s, mccloss - 0.7342, iou_score - 0.2456, accuracy - 0.9227, precision - 0.258
valid: 100%|█| 133/133 [00:01<00:00, 71.15it/s, mccloss - 0.

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



train: 100%|█| 51/51 [00:05<00:00,  8.56it/s, mccloss - 0.3662, iou_score - 0.5427, accuracy - 0.9785, precision - 0.652
valid: 100%|█| 133/133 [00:03<00:00, 41.68it/s, mccloss - 0.8269, iou_score - 0.7825, accuracy - 0.9927, precision - 0.9
No improvement in 2 epochs. Model not saved.

Epoch: 14
train: 100%|█| 51/51 [00:05<00:00,  8.86it/s, mccloss - 0.3449, iou_score - 0.5465, accuracy - 0.9803, precision - 0.672
valid: 100%|█| 133/133 [00:03<00:00, 40.56it/s, mccloss - 0.82, iou_score - 0.8002, accuracy - 0.9933, precision - 0.954
No improvement in 3 epochs. Model not saved.

Epoch: 15
train: 100%|█| 51/51 [00:05<00:00,  9.00it/s, mccloss - 0.3614, iou_score - 0.5255, accuracy - 0.9792, precision - 0.651
valid: 100%|█| 133/133 [00:03<00:00, 34.78it/s, mccloss - 0.7887, iou_score - 0.8255, accuracy - 0.9927, precision - 0.8
No improvement in 2 epochs. Model not saved.

Epoch: 22
train: 100%|█| 51/51 [00:05<00:00,  8.69it/s, mccloss - 0.3034, iou_score - 0.6041, accuracy - 0.9838, pre

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



valid: 100%|█| 133/133 [00:03<00:00, 40.49it/s, mccloss - 0.7753, iou_score - 0.7279, accuracy - 0.9908, precision - 0.9
Model saved!
Decrease decoder learning rate by factor of 10

Epoch: 16
train: 100%|█| 51/51 [00:05<00:00,  9.17it/s, mccloss - 0.3922, iou_score - 0.4728, accuracy - 0.9853, precision - 0.698
valid: 100%|█| 133/133 [00:03<00:00, 37.25it/s, mccloss - 0.7628, iou_score - 0.7312, accuracy - 0.9909, precision - 0.8
Model saved!

Epoch: 17
train: 100%|█| 51/51 [00:05<00:00,  9.36it/s, mccloss - 0.3657, iou_score - 0.503, accuracy - 0.9867, precision - 0.6907
valid: 100%|█| 133/133 [00:03<00:00, 42.48it/s, mccloss - 0.7483, iou_score - 0.7197, accuracy - 0.9908, precision - 0.8
No improvement in 1 epochs. Model not saved.

Epoch: 18
train: 100%|█| 51/51 [00:05<00:00,  8.72it/s, mccloss - 0.3597, iou_score - 0.5108, accuracy - 0.987, precision - 0.6966
valid: 100%|█| 133/133 [00:03<00:00, 39.46it/s, mccloss - 0.7459, iou_score - 0.7461, accuracy - 0.9909, precision - 0.8
Mo

...

train: 100%|█| 51/51 [00:07<00:00,  7.05it/s, mccloss - 0.8573, iou_score - 0.08125, accuracy - 0.64, precision - 0.0815
valid: 100%|█| 133/133 [00:03<00:00, 35.49it/s, mccloss - 0.9445, iou_score - 0.04031, accuracy - 0.8085, precision - 0.
Model saved!

Epoch: 3
train: 100%|█| 51/51 [00:07<00:00,  7.03it/s, mccloss - 0.8075, iou_score - 0.1432, accuracy - 0.7998, precision - 0.144
valid: 100%|█| 133/133 [00:03<00:00, 35.36it/s, mccloss - 0.9307, iou_score - 0.06579, accuracy - 0.9181, precision - 0.
Model saved!

Epoch: 4
train: 100%|█| 51/51 [00:07<00:00,  7.11it/s, mccloss - 0.7786, iou_score - 0.1976, accuracy - 0.8674, precision - 0.199
train: 100%|█| 51/51 [00:07<00:00,  6.58it/s, mccloss - 0.5154, iou_score - 0.4722, accuracy - 0.9699, precision - 0.516
valid: 100%|█| 133/133 [00:04<00:00, 32.63it/s, mccloss - 0.8344, iou_score - 0.6268, accuracy - 0.9865, precision - 0.6
Model saved!

Epoch: 11
train: 100%|█| 51/51 [00:07<00:00,  6.51it/s, mccloss - 0.478, iou_score - 0.5085, 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



valid: 100%|█| 133/133 [00:04<00:00, 28.50it/s, mccloss - 0.8167, iou_score - 0.7238, accuracy - 0.9923, precision - 0.8
No improvement in 1 epochs. Model not saved.

Epoch: 17
train: 100%|█| 51/51 [00:09<00:00,  5.13it/s, mccloss - 0.3915, iou_score - 0.5733, accuracy - 0.9848, precision - 0.669
valid: 100%|█| 133/133 [00:04<00:00, 31.48it/s, mccloss - 0.8156, iou_score - 0.7254, accuracy - 0.9926, precision - 0.8
No improvement in 2 epochs. Model not saved.

Epoch: 18
train: 100%|█| 51/51 [00:09<00:00,  5.22it/s, mccloss - 0.3792, iou_score - 0.5917, accuracy - 0.9858, precision - 0.676
valid: 100%|█| 133/133 [00:04<00:00, 30.88it/s, mccloss - 0.8137, iou_score - 0.7266, accuracy - 0.9924, precision - 0.8
No improvement in 3 epochs. Model not saved.

Epoch: 19
train: 100%|█| 51/51 [00:10<00:00,  4.99it/s, mccloss - 0.3939, iou_score - 0.5755, accuracy - 0.9845, precision - 0.672
valid: 100%|█| 133/133 [00:04<00:00, 28.07it/s, mccloss - 0.8141, iou_score - 0.7334, accuracy - 0.9924, p

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Model saved!

Epoch: 18
train: 100%|█| 51/51 [00:13<00:00,  3.79it/s, mccloss - 0.4014, iou_score - 0.6128, accuracy - 0.9892, precision - 0.677
valid: 100%|█| 133/133 [00:04<00:00, 32.42it/s, mccloss - 0.8172, iou_score - 0.8024, accuracy - 0.9933, precision - 0.8
No improvement in 1 epochs. Model not saved.

Epoch: 19
train: 100%|█| 51/51 [00:14<00:00,  3.57it/s, mccloss - 0.3835, iou_score - 0.6386, accuracy - 0.9901, precision - 0.695
valid: 100%|█| 133/133 [00:04<00:00, 31.25it/s, mccloss - 0.8141, iou_score - 0.7582, accuracy - 0.9932, precision - 0.8
No improvement in 5 epochs. Model not saved.
More than 5 epochs without validation improvement while learning rate <= 1e-5 and training improvement < 0.05...ending training


----------------------------------------------------------
['shade', 'ndvi', 'dsm']

Epoch: 1
train: 100%|█| 51/51 [00:13<00:00,  3.76it/s, mccloss - 0.8696, iou_score - 0.1314, accuracy - 0.8874, precision - 0.135
valid: 100%|█| 133/133 [00:04<00:00, 31.80it/s

In [27]:
print('All processing complete!')

All processing complete!
